# Boxkite demo

This demo shows an end-to-end model training.

## Train a diabetes model

First we import our dependencie.

In [34]:
import os
import pickle
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from boxkite.monitoring.service import ModelMonitoringService

Create the directory to store the model and histogram.

In [35]:
if not os.path.exists('data'):
    os.mkdir('data')
    
model_file = os.path.join('data', 'model.pkl')
histogram_file = os.path.join('data', 'histogram.txt')

Load the diabetes dataset and display the description of the dataset, e.g., 10 attributes and 1 target.

In [8]:
bunch = load_diabetes()

print(bunch.DESCR)

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

Show the training data for the 10 attributes and 1 target respectively.

In [9]:
x_train, x_test, y_train, y_test = train_test_split(
    bunch.data, bunch.target
)

print("The first row is", x_train[0])
print("The prediction is", y_train[0])

The first row is [-0.05273755 -0.04464164 -0.00081689 -0.02632783  0.01081462  0.00714113
  0.0486401  -0.03949338 -0.03581673  0.01963284]
The prediction is 113.0


Train a linear regression model and save it in the local directory.

In [10]:
model = LinearRegression()
model.fit(x_train, y_train)

print("Score: %.2f" % model.score(x_test, y_test))
with open(model_file, "wb") as f:
    pickle.dump(model, f)

Score: 0.47


Save the baseline histogram in the local directory.

In [11]:
# features = [("age", [33, 23, 54, ...]), ("sex", [0, 1, 0]), ...]
features = zip(*[bunch.feature_names, x_train.T])

y_pred = model.predict(x_test)
inference = list(y_pred)

ModelMonitoringService.export_text(
    features=features, inference=inference, path=histogram_file,
)

Verify the model and histogram have been saved in the data directory.

In [27]:
!ls data

histogram.txt model.pkl


## Serve a diabetes model only

First we import our dependencie.

In [36]:
import pickle
from flask import Flask, request
from boxkite.monitoring.collector import BaselineMetricCollector
from boxkite.monitoring.service import ModelMonitoringService

Load the model and histogram from the local directory.

In [37]:
print("Loading model and histogram from local files")

with open(model_file, "rb") as f:
    model = pickle.load(f)

monitor = ModelMonitoringService(
    baseline_collector=BaselineMetricCollector(path=histogram_file)
)

Loading model and histogram from local files


Create the server to serve the model.

In [38]:
app = Flask(__name__)

@app.route("/", methods=["POST"])
def predict():
    features = request.json
    score = model.predict([features])[0]
    pid = monitor.log_prediction(
        request_body=request.data,
        features=features,
        output=score,
    )
    return {"result": score, "prediction_id": pid}


@app.route("/metrics", methods=["GET"])
def metrics():
    return monitor.export_http()[0]

Start the server. By stopping the server, tap "i" twice in the below console.

In [40]:
app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Dec/2021 18:21:00] "GET / HTTP/1.1" 405 -
127.0.0.1 - - [21/Dec/2021 18:21:07] "GET /metrics HTTP/1.1" 200 -


## Serve a diabetes model with monitoring

In [16]:
!ls

README.md          data               deployment         metrics
app                demo.ipynb         docker-compose.yml


Execute docker-compose.yml to start one app server, prometheus and grafana. 

In [17]:
!cat docker-compose.yml

version: "3"
services:
  graf:
    image: grafana/grafana:7.5.3
    ports:
      - 3000:3000
    volumes:
      - ./metrics/provisioning:/etc/grafana/provisioning
      - ./metrics/dashboards:/var/lib/grafana/dashboards
  prom:
    image: prom/prometheus:v2.26.0
    ports:
      - 9090:9090
    volumes:
      - ./metrics/prometheus.yml:/etc/prometheus/prometheus.yml
  app:
    build:
      context: app
    ports:
      - 5000:5000
    volumes:
      - ./data/model.pkl:/app/model.pkl
      - ./data/histogram.txt:/app/histogram.txt


In [18]:
!docker-compose up -d

Creating network "ha-boxkite-notebook_default" with the default driver
Building app
Step 1/7 : FROM python:3.8-slim
 ---> 1e46b5746c7c
Step 2/7 : WORKDIR /app
 ---> Using cache
 ---> 8885f2557874
Step 3/7 : COPY requirements.txt /app/requirements.txt
 ---> Using cache
 ---> 7949bc115878
Step 4/7 : RUN pip install --no-cache-dir -r requirements.txt
 ---> Using cache
 ---> 2ebb5f5fceea
Step 5/7 : COPY serve.py /app/serve.py
 ---> Using cache
 ---> 378c9fd0131d
Step 6/7 : ENV FLASK_APP serve.py
 ---> Using cache
 ---> f6b8fe5da06d
Step 7/7 : CMD ["flask", "run", "--host=0.0.0.0"]
 ---> Using cache
 ---> de66c03a9004
Successfully built de66c03a9004
Successfully tagged ha-boxkite-notebook_app:latest
Creating ha-boxkite-notebook_graf_1 ... 
Creating ha-boxkite-notebook_prom_1 ... 
Creating ha-boxkite-notebook_app_1  ... 
ting ha-boxkite-notebook_prom_1 ... done

Show the running containers.

In [22]:
!docker ps

CONTAINER ID        IMAGE                     COMMAND                  CREATED             STATUS              PORTS                    NAMES
a103162ea19e        ha-boxkite-notebook_app   "flask run --host=0.…"   2 minutes ago       Up 2 minutes        0.0.0.0:5000->5000/tcp   ha-boxkite-notebook_app_1
53059760c0dc        grafana/grafana:7.5.3     "/run.sh"                2 minutes ago       Up 2 minutes        0.0.0.0:3000->3000/tcp   ha-boxkite-notebook_graf_1
505a3b4114d7        prom/prometheus:v2.26.0   "/bin/prometheus --c…"   2 minutes ago       Up 2 minutes        0.0.0.0:9090->9090/tcp   ha-boxkite-notebook_prom_1


Check if the app server is serving the metrics of histogram.

In [23]:
!curl -s http://localhost:5000/metrics

# HELP feature_0_value_baseline Baseline values for feature: age
# TYPE feature_0_value_baseline histogram
feature_0_value_baseline_bucket{le="-0.107225631607358"} 3.0
feature_0_value_baseline_bucket{le="-0.0854304009012407"} 16.0
feature_0_value_baseline_bucket{le="-0.06363517019512341"} 35.0
feature_0_value_baseline_bucket{le="-0.041839939489006106"} 73.0
feature_0_value_baseline_bucket{le="-0.020044708782888804"} 115.0
feature_0_value_baseline_bucket{le="0.0017505219232284985"} 153.0
feature_0_value_baseline_bucket{le="0.023545752629345787"} 210.0
feature_0_value_baseline_bucket{le="0.04534098333546309"} 262.0
feature_0_value_baseline_bucket{le="0.06713621404158039"} 302.0
feature_0_value_baseline_bucket{le="0.0889314447476977"} 327.0
feature_0_value_baseline_bucket{le="0.110726675453815"} 331.0
feature_0_value_baseline_bucket{le="+Inf"} 331.0
# HELP feature_1_value_baseline_total Baseline values for feature: sex
# TYPE feature_1_value_baseline_total counter
feature_1_value_baseline

Import the dependencies in order to simulate the dummy traffic.

In [24]:
from random import gauss
from time import sleep
from requests import Session

Generate the online queries.

In [25]:
data = [0.03, 0.0506801187398187, -0.002, -0.01, 0.04, 0.01, 0.08, -0.04, 0.005, -0.1]

with Session() as s:
    for i in range(60):
        data[0] = gauss(6, 2)
        resp = s.post("http://localhost:5000", json=data)
        resp.raise_for_status()
        if i % 10 == 0:
            print(f"response[{i}]: {resp.json()}")
        sleep(0.1)


response[0]: {'prediction_id': 'unknown-server/2021-12-21T17:02:13/2eaea7e6-6f8f-4f0e-b6e6-11c69e6f3973', 'result': 89.08273298697215}
response[10]: {'prediction_id': 'unknown-server/2021-12-21T17:02:15/cb7f0281-2d6d-432c-acb2-e1cd03776180', 'result': 90.6808637814857}
response[20]: {'prediction_id': 'unknown-server/2021-12-21T17:02:16/e05c43dd-53dc-425c-afe6-b9c0d1a68116', 'result': 86.1178976556911}
response[30]: {'prediction_id': 'unknown-server/2021-12-21T17:02:17/010b0ae3-0dbf-4479-889d-0c313744f845', 'result': 99.79033532280393}
response[40]: {'prediction_id': 'unknown-server/2021-12-21T17:02:18/0416c969-2ccc-4851-aa55-9dc767b738c6', 'result': 84.96351431564621}
response[50]: {'prediction_id': 'unknown-server/2021-12-21T17:02:19/ca1d23dd-03ed-4acd-a30d-2ef36df83fcc', 'result': 89.77162032381283}


Go to [grafana dashboard](http://localhost:3000/) to check the visuals.

![grafana dashboard](./pix/grafana.png)

Go to [prometheus](http://localhost:9090/) to check the metrics.

![prometheus](./pix/prometheus.png)

Shutdown the servers and remove the docker containers.

In [26]:
!docker-compose down

!docker image rm grafana/grafana:7.5.3
!docker image rm prom/prometheus:v2.26.0
!docker image rm ha-boxkite-notebook_app

Stopping ha-boxkite-notebook_app_1  ... 
Stopping ha-boxkite-notebook_graf_1 ... 
Stopping ha-boxkite-notebook_prom_1 ... 
Removing ha-boxkite-notebook_app_1  ... mdone
Removing ha-boxkite-notebook_graf_1 ... 
Removing ha-boxkite-notebook_prom_1 ... 
Removing network ha-boxkite-notebook_defaulte


## Serve a diabetes model with HA

Start kubernetes on MacOS in a pseudo-distributed mode.

In [41]:
!minikube start

😄  minikube v1.23.0 on Darwin 12.0.1
✨  Using the docker driver based on existing profile
👍  Starting control plane node minikube in cluster minikube
🚜  Pulling base image ...
🔄  Restarting existing docker container for "minikube" ...
🐳  Preparing Kubernetes v1.22.1 on Docker 20.10.8 ...
🔎  Verifying Kubernetes components...
    ▪ Using image gcr.io/k8s-minikube/storage-provisioner:v5
🌟  Enabled addons: storage-provisioner, default-storageclass
🏄  Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default


By deployment.yml, spring up three app servers, each of which has model.pkl and histogram.txt already baked in the image.

In [48]:
!cat deployment/deployment.yml

apiVersion: apps/v1
kind: Deployment
metadata:
  name: ml-deployment
  labels:
    app: ml-server
spec:
  replicas: 3
  selector:
    matchLabels:
      app: ml-server
  template:
    metadata:
      labels:
        app: ml-server
      annotations:
        prometheus.io/scrape: "true"
    spec:
#      volumes:
#        - name: task-pv-storage
#          persistentVolumeClaim:
#            claimName: task-pv-claim
      containers:
        - name: ml-server
          image: tintinrevient/boxkite-app
          ports:
            - containerPort: 5000
#          volumeMounts:
#            - mountPath: "/app"
#              name: task-pv-storage


In [50]:
!kubectl apply -f deployment/deployment.yml

deployment.apps/ml-deployment created


In [51]:
!kubectl get deployment

NAME            READY   UP-TO-DATE   AVAILABLE   AGE
ml-deployment   3/3     3            3           16s


In [52]:
!kubectl rollout status deployment/ml-deployment

deployment "ml-deployment" successfully rolled out


In [53]:
!kubectl get pods

NAME                             READY   STATUS    RESTARTS   AGE
ml-deployment-6bb5867688-9f4s2   1/1     Running   0          20s
ml-deployment-6bb5867688-mlz25   1/1     Running   0          20s
ml-deployment-6bb5867688-mp9w7   1/1     Running   0          20s


In [54]:
!kubectl expose deployment ml-deployment --port=5000 --type=NodePort

service/ml-deployment exposed


In [55]:
!kubectl describe svc ml-deployment

Name:                     ml-deployment
Namespace:                default
Labels:                   app=ml-server
Annotations:              <none>
Selector:                 app=ml-server
Type:                     NodePort
IP Family Policy:         SingleStack
IP Families:              IPv4
IP:                       10.104.27.43
IPs:                      10.104.27.43
Port:                     <unset>  5000/TCP
TargetPort:               5000/TCP
NodePort:                 <unset>  32408/TCP
Endpoints:                172.17.0.3:5000,172.17.0.4:5000,172.17.0.5:5000
Session Affinity:         None
External Traffic Policy:  Cluster
Events:                   <none>


Start the load balancer by minikube on MacOS. By stopping the server, tap "i" twice in the below console.

In [56]:
!minikube service ml-deployment

|-----------|---------------|-------------|---------------------------|
| NAMESPACE |     NAME      | TARGET PORT |            URL            |
|-----------|---------------|-------------|---------------------------|
| default   | ml-deployment |        5000 | http://192.168.49.2:32408 |
|-----------|---------------|-------------|---------------------------|
🏃  Starting tunnel for service ml-deployment.
|-----------|---------------|-------------|------------------------|
| NAMESPACE |     NAME      | TARGET PORT |          URL           |
|-----------|---------------|-------------|------------------------|
| default   | ml-deployment |             | http://127.0.0.1:63654 |
|-----------|---------------|-------------|------------------------|
🎉  Opening service default/ml-deployment in default browser...
❗  Because you are using a Docker driver on darwin, the terminal needs to be open to run it.
^C
✋  Stopping tunnel for service ml-deployment.

❌  Exiting due to SVC_TUNNEL_STOP: stopping

Generate the dummy traffic as done before, only with the port number modified as referenced above.

In [58]:
from random import gauss
from time import sleep
from requests import Session

data = [0.03, 0.0506801187398187, -0.002, -0.01, 0.04, 0.01, 0.08, -0.04, 0.005, -0.1]

with Session() as s:
    for i in range(60):
        data[0] = gauss(6, 2)
        resp = s.post("http://localhost:63654", json=data)
        resp.raise_for_status()
        if i % 10 == 0:
            print(f"response[{i}]: {resp.json()}")
        sleep(0.1)

ConnectionError: HTTPConnectionPool(host='localhost', port=63654): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa7c2935970>: Failed to establish a new connection: [Errno 61] Connection refused'))

Within the cluster, the port number 5000 can be referenced by each server. Below commands can be executed in a new terminal.

In [50]:
!kubectl exec -it ml-deployment-6bb5867688-hmkzj -- bash

root@ml-deployment-6bb5867688-9f4s2:/app# curl -s http://172.17.0.2:5000/metrics
root@ml-deployment-6bb5867688-9f4s2:/app# curl -s http://172.17.0.4:5000/metrics
root@ml-deployment-6bb5867688-9f4s2:/app# curl -s http://172.17.0.5:5000/metrics

root@ml-deployment-6bb5867688-hmkzj:/app# command terminated with exit code 137


### Install Prometheus

Install prometheus on MacOS using [precompiled binaries](https://prometheus.io/docs/prometheus/latest/installation/).

Configure prometheus.yml to point to the load balancer in http://localhost:63654.
```yaml
scrape_configs:
    - job_name: "boxkite"
        static_configs:
          - targets: [ "localhost:63654" ]
```

Start prometheus:
```bash
./prometheus
````

### Install Grafana

Install grafana on MacOS using [standalone binaries](https://grafana.com/docs/grafana/latest/installation/mac/).

Start grafana:
```bash
./bin/grafana-server web
```

Configure the prometheus datasource and import model.json to create the dashboard. 

Shutdown the servers and remove the pods.

In [59]:
!kubectl delete deployment ml-deployment
!kubectl delete svc ml-deployment

deployment.apps "ml-deployment" deleted
service "ml-deployment" deleted


In [60]:
!minikube stop

✋  Stopping node "minikube"  ...
🛑  Powering off "minikube" via SSH ...
🛑  1 nodes stopped.
